# Предисловие
В ходе этого блокнота будут подготовлены все файлы для проведения MM/GBSA, а именно:
*  md_0_10.tpr
*  md_0_10.xtc
*  index.ndx
*  topol.top
Для них, в свою очеред нужны будут соответсвующие файлы, о которых я буду упоминать в соответствующих разделах

# Шаг 0. Проверка входных файлов

Перед проведением работы нужно убедиться, что присутствуют все необходимые файлы, полученные в ходе МД. Пользователю необходимо загрузить в папку 1_MD_results файлы:
* топологии topol.top
    * силовое поле (в текущей версии chrmm36-jul2022.ff)
*  индекс
*  траектории

In [1]:
sourse_folder = "1_MD_results"
frag_params_folder = "2_fragments_parametrs"
result_folder = "3_MMGBSA"
tmp_folder = "tmp"

# Шаг 1. Изменение файла траектории
Файл траектории можно модифицировать основываясь только на файле траетории и информации о расположении атомов в начале. Это самый простой файл в подготовке, все остальные файлы будут нуждаться в промежуточных файлах из этого этапа.

**Файлы на вход:**
* md_0_10.xtc
* md_0_10.gro

*Опционалоно:*
* ligand.pdb файл, в котором указана правильная связанность разбиваемого лиганда
* smiles.smi информаци для MedChemSplitter о том, на какие фрагменты разбиравть лиганд

**На выходе:**
* frame_0_with_frags.gro
* traj_with_frags.xtc

*Для других шагов:*
* fr1.pdb,...,fri.pdb файлы всех фрагментов, которые нужны для подготовки параметров этих фрагментов, которые нужны для файла топологии
* frame_0_with_frags.gro файл, который понадобится для создания tpr файла


## Логика выполнения:
1. Разбиваем траекторию на отдельные gro-файлы. Далее для каждого кадра:
2. Выделяем лиганд в pdb-файл
3. Делим лиганд на фрагменты в **MedChemSplitter**
4. Обрабатываем полученные pdb-файлы фрагментов
5. Вставляем фрагменты в gro-файл кадра
6. Когда все кадры обработаны, собираем их обратно в файл траетории

In [70]:
import mdtraj as md
import os
import re
import openbabel

In [71]:
xtc_in = sourse_folder + '/' + 'md_0_10.xtc'
top_in = sourse_folder + '/' + 'md_0_10.gro'
gro_folder = tmp_folder + '/' + 'frames'
modif_folder = tmp_folder + '/' + 'frames_modif'


ligand_resname = 'JZ4'

In [72]:
# Загружаем траекторию XTC с топологией (например, PDB или GRO)
traj = md.load(xtc_in, top=top_in)
os.makedirs(gro_folder, exist_ok=True)

# Разбиение траектории на отдельные кадры и сохранение их в формате GRO
for i in range(traj.n_frames):
    traj[i].save(f'{gro_folder}/frame_{i}.gro')

In [73]:
# Получаем список всех файлов в папке
gro_files = [f for f in os.listdir(gro_folder) if f.endswith('.gro')]

# Сортируем файлы с учетом числовой части в их именах
gro_files.sort(key=lambda f: int(re.search(r'(\d+)', f).group()))

# Проверяем, что файлы существуют
if not gro_files:
    raise ValueError("Нет файлов .gro в указанной папке!")

# Выводим порядок файлов GRO
print("Используемые GRO файлы:")
for gro_file in gro_files:
    print(gro_file)

Используемые GRO файлы:
frame_0.gro
frame_1.gro
frame_2.gro
frame_3.gro
frame_4.gro
frame_5.gro
frame_6.gro
frame_7.gro
frame_8.gro
frame_9.gro
frame_10.gro


In [74]:
def fragment_extraction(ligand_resname, frame_number, traj, tmp_folder):
    # Извлекает фрагменты из траектории в формате pdb

    # Получаем инфдексы лиганда в молекуле
    ligand_indices = traj.topology.select(f'resname {ligand_resname}')
    ligand_traj = traj.atom_slice(ligand_indices)
    
    currend_wd = f'{tmp_folder}/pdb/frame_{frame_number}'
    ligand_without_bonds = f'{currend_wd}/ligand_without_bonds.pdb'
    ligand_pdb = f"{tmp_folder}/ligand.pdb"  # Файл с правильными связями
    output_pdb = f"{currend_wd}/ligand_final.pdb"  # Итоговый файл
    splitter = f"{tmp_folder}/medchem_fragment_splitter/medchem_fragment_splitter.py"
    
    # Сохраняем лиганд в PDB файл
    os.makedirs(currend_wd, exist_ok=True)
    ligand_traj.save(ligand_without_bonds)
        
    copy_conect_and_renumber(ligand_pdb, ligand_without_bonds, output_pdb)
    
   
    os.system(f'python {splitter} -l {output_pdb} -w {currend_wd} -p -x -a -f pdb -o result')

    fragment_files = [f for f in os.listdir(currend_wd) if f.endswith('.pdb') and not f in ['ligand_final.pdb', 'ligand_without_bonds.pdb'] and not f.endswith('converted.pdb')]
    fragment_files.sort(key=lambda f: int(re.search(r'(\d+)', f).group()))
    return fragment_files

In [75]:
import os
import subprocess
from rdkit import Chem
from rdkit.Chem import Draw

import matplotlib.pyplot as plt

def copy_conect_and_renumber(ligand_pdb, ligand_without_bonds, output_pdb):
    """
    Копирует блок CONECT из ligand.pdb в ligand_without_bonds.pdb и перенумеровывает атомы.
    """
    if not os.path.exists(ligand_pdb):
        print(f"{ligand_pdb} не найден. Генерируем с помощью Open Babel...")
        subprocess.run(["obabel", ligand_without_bonds, "-O", ligand_pdb, "-p", "7"], check=True)
        visualize_molecule_2D(ligand_pdb)

    # Читаем блок CONECT из ligand.pdb
    conect_lines = []
    with open(ligand_pdb, "r") as f:
        for line in f:
            if line.startswith("CONECT"):
                conect_lines.append(line)

    if not conect_lines:
        print("Ошибка: В ligand.pdb нет записей CONECT!")
        return

    # Читаем атомы из ligand_without_bonds.pdb
    atom_lines = []
    with open(ligand_without_bonds, "r") as f:
        for line in f:
            if line.startswith(("ATOM", "HETATM")):
                atom_lines.append(line)

    # Создаём отображение старых номеров атомов в новые
    old_to_new_index = {}
    new_atom_lines = []
    
    for new_index, line in enumerate(atom_lines, start=1):
        old_index = int(line[6:11])  # Берем старый номер атома
        old_to_new_index[old_index] = new_index  # Сопоставляем старый номер с новым
        new_line = f"{line[:6]}{new_index:5}{line[11:]}"  # Заменяем номер в строке
        new_atom_lines.append(new_line)

    # Читаем блок CONECT из ligand.pdb
    conect_lines = []
    with open(ligand_pdb, "r") as f:
        for line in f:
            if line.startswith("CONECT"):
                conect_lines.append(line)

    if not conect_lines:
        print("Ошибка: В ligand.pdb нет записей CONECT!")
        return
        
    # Создаём новый PDB-файл
    with open(output_pdb, "w") as f:
        f.writelines(new_atom_lines)  # Записываем атомы с новыми номерами
        f.writelines(conect_lines)  # Добавляем CONECT
        f.write("END\n")  # Завершаем PDB-файл

    print(f"Файл {output_pdb} сохранен: перенумерованы атомы + добавлен блок CONECT.")

def visualize_molecule_2D(ligand_pdb):
    """
    Загружает PDB-файл в RDKit, переводит в 2D и строит изображение молекулы.
    """
    mol = Chem.MolFromPDBFile(ligand_pdb, removeHs=False)
    if mol is None:
        print("Ошибка: не удалось загрузить PDB-файл в RDKit.")
        return

    # Генерируем 2D-координаты
    mol = Chem.MolFromSmiles(Chem.MolToSmiles(mol))

    # Визуализируем молекулу в 2D
    img = Draw.MolToImage(mol, size=(500, 500))
    
    # Показываем изображение
    plt.figure(figsize=(5, 5))
    plt.imshow(img)
    plt.axis("off")
    plt.show()

In [76]:
def fragment_prepare(input_pdb, fragment_number, output_pdb = None):
    
    if output_pdb == None:
        output_pdb = input_pdb[:-4] + '_converted.pdb'  # Новый PDB файл
        
    # Создание объекта для конвертации
    ob_conversion = openbabel.OBConversion()
    ob_conversion.SetInAndOutFormats("pdb", "pdb")
        
    # Чтение молекулы из PDB файла
    mol = openbabel.OBMol()
    ob_conversion.ReadFile(mol, input_pdb)
        
    # Проходим по атомам и меняем название остатка и номер
    for atom in openbabel.OBMolAtomIter(mol):
        residue = atom.GetResidue()
        residue.SetName(f"FR{fragment_number+1}")
        residue.SetNum(1)
        
    # Сохраняем измененный файл PDB
    ob_conversion.WriteFile(mol, output_pdb)
        
    print(f"Файл с изменениями сохранен как {output_pdb}")
    
    os.system(f'gmx editconf -f {output_pdb} -o {input_pdb[:-4]}.gro')

In [77]:
os.makedirs('pdb', exist_ok=True)

for i in range(len(gro_files)):
    # Создаем папки для работы с обработанными файлами
    os.makedirs(f'{tmp_folder}/frames_modif', exist_ok=True)

    # Загружаем траекторию, выделяем лиганд, получаем файлы фрагментов
    traj = md.load(f'{gro_folder}/{gro_files[i]}')
    fragment_files = fragment_extraction(ligand_resname, i, traj, tmp_folder)
    
    print(fragment_files)

    currend_wd = f'{tmp_folder}/pdb/frame_{i}'
    
    for j in range(len(fragment_files)):
        # Файлы для чтения и записи
        input_pdb = f"{currend_wd}/{fragment_files[j]}"  # Ваш исходный PDB файл
        fragment_prepare(input_pdb, j) #Предобработка файлов фрагментов
               
        
        # Пути к исходному и целевому файлам
        source_file_path = f'{input_pdb[:-4]}.gro'  # Исходный файл, из которого считываем строки
        if j == 0:
            target_file_path = f'{tmp_folder}/frames/frame_{i}.gro'  # Целевой файл, в который вставляем строки
        else:
            target_file_path = f'{tmp_folder}/frames_modif/frame_modif_{i}.gro'
        new_file_path = f'{tmp_folder}/frames_modif/frame_modif_{i}.gro'  # Новый файл, куда будем записывать результат
        
        # Открываем исходный файл для чтения
        with open(source_file_path, 'r') as source_file:
            # Чтение всех строк из исходного файла
            lines = source_file.readlines()
        
            # Извлекаем число строк, которые нужно прочитать (вторая строка)
            num_lines_to_read = int(lines[1].strip())
        
            # Считываем нужное количество строк, начиная с 3-й строки
            lines_to_add = [lines[i] for i in range(2, 2 + num_lines_to_read)]
        
        # Открываем целевой файл для чтения
        with open(target_file_path, 'r') as target_file:
            # Чтение всех строк из целевого файла
            target_lines = target_file.readlines()
        
        # Заменяем вторую строку
        target_lines[1] = str(int(target_lines[1]) + len(lines_to_add)) + '\n'  # Укажите, что хотите поставить в качестве второй строки
    
        # Поиск последней строки, содержащей 'JZ4'
        last_jz4_index = -1
        
        for k, line in enumerate(target_lines):
            if "JZ4" in line or f'FR{j}' in line:
                last_jz4_index = k
        print(f"for FR{j+1} last_jz4_index = {last_jz4_index}")
        
        # Вставляем строки перед последней строкой
        target_lines = target_lines[:last_jz4_index+1] + [line for line in lines_to_add] + target_lines[last_jz4_index+1:]
        
        # Записываем результат в новый файл
        with open(new_file_path, 'w') as new_target_file:
            # Записываем измененные строки в новый файл
            new_target_file.writelines(target_lines)
        print(f"FR{j+1} addet to {new_file_path}")
        
        os.system(f'gmx genconf -f {new_file_path} -o {new_file_path[:-4]}.gro -renumber')
        print("------------------------------------------------")

Файл tmp/pdb/frame_0/ligand_final.pdb сохранен: перенумерованы атомы + добавлен блок CONECT.
After file reading: 22 atoms total, 12 H
At processing_molecules start: 22 atoms total, 12 H
At start of splitting: 22 atoms total, 12 H
molecule was splitted sucsessefuly
2
10
Property Updated
H added
12
Property Updated
H added
fragments was sanitized sucsessefuly
At processing_molecules start: 10 atoms total, 0 H
At start of splitting: 10 atoms total, 0 H
molecule was splitted sucsessefuly
2
3
Property Updated
H added
7
Property Updated
H added
fragments was sanitized sucsessefuly
['result_1.pdb', 'result_2.pdb']
Файл с изменениями сохранен как tmp/pdb/frame_0/result_1_converted.pdb
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 11 atoms
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.
for FR1 last_jz4_index = 2637
FR1 addet to tmp/frames_modif/fram

/home/daniil/miniconda3/envs/md-intro-tutorial/lib/python3.12/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
               :-) GROMACS - gmx editconf, 2024.3-conda_forge (-:

Executable:   /home/daniil/miniconda3/envs/md-intro-tutorial/bin.AVX2_256/gmx
Data prefix:  /home/daniil/miniconda3/envs/md-intro-tutorial
Working dir:  /home/daniil/MD/MM_GBSA/With_FR1/start2end
Command line:
  gmx editconf -f tmp/pdb/frame_0/result_1_converted.pdb -o tmp/pdb/frame_0/result_1.gro


Back Off! I just backed up tmp/pdb/frame_0/result_1.gro to tmp/pdb/frame_0/#result_1.gro.3#

GROMACS reminds you: "Science, my lad, is made up of mistakes, but they are mistakes which it is useful to make, because they lead little by little to the truth." (Jules Verne)

               :-) GROMACS - gmx genconf, 2024.3-conda_forge (-:

E

------------------------------------------------
Файл с изменениями сохранен как tmp/pdb/frame_0/result_2_converted.pdb
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 13 atoms
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.
for FR2 last_jz4_index = 2648
FR2 addet to tmp/frames_modif/frame_modif_0.gro
------------------------------------------------


               :-) GROMACS - gmx genconf, 2024.3-conda_forge (-:

Executable:   /home/daniil/miniconda3/envs/md-intro-tutorial/bin.AVX2_256/gmx
Data prefix:  /home/daniil/miniconda3/envs/md-intro-tutorial
Working dir:  /home/daniil/MD/MM_GBSA/With_FR1/start2end
Command line:
  gmx genconf -f tmp/frames_modif/frame_modif_0.gro -o tmp/frames_modif/frame_modif_0.gro -renumber


Back Off! I just backed up tmp/frames_modif/frame_modif_0.gro to tmp/frames_modif/#frame_modif_0.gro.2#

GROMACS reminds you: "Science, my lad, is made up of mistakes, but they are mistakes which it is useful to make, because they lead little by little to the truth." (Jules Verne)



Файл tmp/pdb/frame_1/ligand_final.pdb сохранен: перенумерованы атомы + добавлен блок CONECT.
After file reading: 22 atoms total, 12 H
At processing_molecules start: 22 atoms total, 12 H
At start of splitting: 22 atoms total, 12 H
molecule was splitted sucsessefuly
2
10
Property Updated
H added
12
Property Updated
H added
fragments was sanitized sucsessefuly
At processing_molecules start: 10 atoms total, 0 H
At start of splitting: 10 atoms total, 0 H
molecule was splitted sucsessefuly
2
3
Property Updated
H added
7
Property Updated
H added
fragments was sanitized sucsessefuly
['result_1.pdb', 'result_2.pdb']
Файл с изменениями сохранен как tmp/pdb/frame_1/result_1_converted.pdb
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 11 atoms
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.
for FR1 last_jz4_index = 2637
FR1 addet to tmp/frames_modif/fram

...

               :-) GROMACS - gmx genconf, 2024.3-conda_forge (-:

Executable:   /home/daniil/miniconda3/envs/md-intro-tutorial/bin.AVX2_256/gmx
Data prefix:  /home/daniil/miniconda3/envs/md-intro-tutorial
Working dir:  /home/daniil/MD/MM_GBSA/With_FR1/start2end
Command line:
  gmx genconf -f tmp/frames_modif/frame_modif_9.gro -o tmp/frames_modif/frame_modif_9.gro -renumber


Back Off! I just backed up tmp/frames_modif/frame_modif_9.gro to tmp/frames_modif/#frame_modif_9.gro.2#

GROMACS reminds you: "I Quit My Job Blowing Leaves" (Beck)



------------------------------------------------
Файл tmp/pdb/frame_10/ligand_final.pdb сохранен: перенумерованы атомы + добавлен блок CONECT.
After file reading: 22 atoms total, 12 H
At processing_molecules start: 22 atoms total, 12 H
At start of splitting: 22 atoms total, 12 H
molecule was splitted sucsessefuly
2
10
Property Updated
H added
12
Property Updated
H added
fragments was sanitized sucsessefuly
At processing_molecules start: 10 atoms total, 0 H
At start of splitting: 10 atoms total, 0 H
molecule was splitted sucsessefuly
2
3
Property Updated
H added
7
Property Updated
H added
fragments was sanitized sucsessefuly
['result_1.pdb', 'result_2.pdb']
Файл с изменениями сохранен как tmp/pdb/frame_10/result_1_converted.pdb
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 11 atoms
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.
for FR1 last_

               :-) GROMACS - gmx editconf, 2024.3-conda_forge (-:

Executable:   /home/daniil/miniconda3/envs/md-intro-tutorial/bin.AVX2_256/gmx
Data prefix:  /home/daniil/miniconda3/envs/md-intro-tutorial
Working dir:  /home/daniil/MD/MM_GBSA/With_FR1/start2end
Command line:
  gmx editconf -f tmp/pdb/frame_10/result_1_converted.pdb -o tmp/pdb/frame_10/result_1.gro


Back Off! I just backed up tmp/pdb/frame_10/result_1.gro to tmp/pdb/frame_10/#result_1.gro.3#

GROMACS reminds you: "It's easy to remember: a half a kT is equal to five fourths of a kJ/mol." (Anders Gabrielsson)

               :-) GROMACS - gmx genconf, 2024.3-conda_forge (-:

Executable:   /home/daniil/miniconda3/envs/md-intro-tutorial/bin.AVX2_256/gmx
Data prefix:  /home/daniil/miniconda3/envs/md-intro-tutorial
Working dir:  /home/daniil/MD/MM_GBSA/With_FR1/start2end
Command line:
  gmx genconf -f tmp/frames_modif/frame_modif_10.gro -o tmp/frames_modif/frame_modif_10.gro -renumber


Back Off! I just backed up tmp/frames

------------------------------------------------
Файл с изменениями сохранен как tmp/pdb/frame_10/result_2_converted.pdb
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 13 atoms
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.
for FR2 last_jz4_index = 2648
FR2 addet to tmp/frames_modif/frame_modif_10.gro
------------------------------------------------



GROMACS reminds you: "It's easy to remember: a half a kT is equal to five fourths of a kJ/mol." (Anders Gabrielsson)

               :-) GROMACS - gmx editconf, 2024.3-conda_forge (-:

Executable:   /home/daniil/miniconda3/envs/md-intro-tutorial/bin.AVX2_256/gmx
Data prefix:  /home/daniil/miniconda3/envs/md-intro-tutorial
Working dir:  /home/daniil/MD/MM_GBSA/With_FR1/start2end
Command line:
  gmx editconf -f tmp/pdb/frame_10/result_2_converted.pdb -o tmp/pdb/frame_10/result_2.gro


Back Off! I just backed up tmp/pdb/frame_10/result_2.gro to tmp/pdb/frame_10/#result_2.gro.3#

GROMACS reminds you: "It's easy to remember: a half a kT is equal to five fourths of a kJ/mol." (Anders Gabrielsson)

               :-) GROMACS - gmx genconf, 2024.3-conda_forge (-:

Executable:   /home/daniil/miniconda3/envs/md-intro-tutorial/bin.AVX2_256/gmx
Data prefix:  /home/daniil/miniconda3/envs/md-intro-tutorial
Working dir:  /home/daniil/MD/MM_GBSA/With_FR1/start2end
Command line:
  gmx genconf -f tmp/f

In [79]:
# Папка, в которой хранятся файлы GRO
gro_folder = f'{tmp_folder}/frames_modif'  # Замените на вашу папку с файлами

# Получаем список всех файлов в папке
gro_files = [f for f in os.listdir(gro_folder) if f.endswith('.gro')]

# Сортируем файлы с учетом числовой части в их именах
gro_files.sort(key=lambda f: int(re.search(r'(\d+)', f).group()))

# Проверяем, что файлы существуют
if not gro_files:
    raise ValueError("Нет файлов .gro в указанной папке!")

# Выводим порядок файлов GRO
print("Используемые GRO файлы:")
for gro_file in gro_files:
    print(gro_file)

# Загружаем кадры из всех найденных файлов GRO
frames = []
for gro_file in gro_files:
    # Загружаем кадр из каждого файла GRO
    frame = md.load(os.path.join(gro_folder, gro_file), top=f'{gro_folder}/frame_modif_0.gro')  # Указываем топологию
    frames.append(frame)  # Добавляем кадр в список

# Объединяем все кадры в одну траекторию
combined_traj = md.join(frames)

# Сохраняем объединенную траекторию в формате XTC
combined_traj.save(f'{tmp_folder}/combined_modif_traj.xtc')

# Выводим сообщение о завершении
print("\nОбъединенная траектория сохранена в 'modif_traj.xtc'")

Используемые GRO файлы:
frame_modif_0.gro
frame_modif_1.gro
frame_modif_2.gro
frame_modif_3.gro
frame_modif_4.gro
frame_modif_5.gro
frame_modif_6.gro
frame_modif_7.gro
frame_modif_8.gro
frame_modif_9.gro
frame_modif_10.gro

Объединенная траектория сохранена в 'modif_traj.xtc'


У нас получен файл траектории. Из-за того, что в ~бухгалтерии~ в **mdtraj** все перепутали, нужно вернуть нормальную типизацию атомов 

In [80]:
def fix_atom_names(original_gro, modified_gro, output_gro, start_line_orig, start_line_mod, num_lines):
    """
    Заменяет имена атомов в `modified_gro`, используя имена из `original_gro`.
    
    Аргументы:
    original_gro   - Исходный файл .gro (где правильные имена атомов).
    modified_gro   - Файл .gro, в который нужно вставить имена атомов.
    output_gro     - Имя выходного файла с исправленными именами атомов.
    start_line_orig - Первая строка в original_gro (откуда берем имена).
    start_line_mod  - Первая строка в modified_gro (куда вставляем имена).
    num_lines      - Количество строк, которые нужно обработать.
    """
    with open(original_gro, "r") as orig_file:
        orig_lines = orig_file.readlines()

    with open(modified_gro, "r") as mod_file:
        mod_lines = mod_file.readlines()

    # Проверяем, что файлов хватает для указанного количества строк
    if len(orig_lines) < start_line_orig + num_lines - 1 or len(mod_lines) < start_line_mod + num_lines - 1:
        raise ValueError("Один из файлов содержит недостаточно строк для обработки.")

    # Переносим имена атомов
    for i in range(num_lines):
        orig_index = start_line_orig - 1 + i  # Индекс строки в original_gro
        mod_index = start_line_mod - 1 + i    # Индекс строки в modified_gro
        
        orig_atom_type = orig_lines[orig_index][9:15]  # Извлекаем имя атома (позиции 9-15)
        mod_lines[mod_index] = mod_lines[mod_index][:9] + orig_atom_type + mod_lines[mod_index][15:]  # Заменяем

    # Записываем исправленный файл
    with open(output_gro, "w") as out_file:
        out_file.writelines(mod_lines)

    print(f"Файл {output_gro} успешно создан с исправленными именами атомов.")

# Используем функцию для исправления
fix_atom_names(
    original_gro,            # Исходный файл с правильными именами атомов
    modified_gro, # Файл, в который вставляем имена
    output_gro, # Выходной исправленный файл
    start_line_orig=3,  # Первая строка в md_0_10.gro
    start_line_mod=3,   # Первая строка в frame_modif_0_renum.gro
    num_lines= 2619     # Количество строк для обработки (до строки с фрагментами)
)

fix_atom_names(
    original_gro,            # Исходный файл с правильными именами атомов
    output_gro, # Файл, в который вставляем имена
    output_gro, # Выходной исправленный файл
    start_line_orig=2639,  # Первая строка в md_0_10.gro
    start_line_mod=2663,   # Первая строка в frame_modif_0_renum.gro
    num_lines=33532-2663      # Количество строк для обработки
)

Файл tmp/fixed_frame_modif_0.gro успешно создан с исправленными именами атомов.
Файл tmp/fixed_frame_modif_0.gro успешно создан с исправленными именами атомов.


## Итог:

У нас готовы файлы:
* Траектории
* Координат в 0 кадре траетории
* pdb-файлы фрагментов, которые можно использовать, чтобы получить параметры для них

# Шаг 2: Подготовка файла индексов
Поскольку у нас появились новые молекулы, нам нужно отразить их в файле индексов (.ndx)
Также, нам нужно создать в нем группу, в которой будут объединены белок, лиганд и фрагменты (эта группа пригодится при создании tpr-файла)

In [56]:
import subprocess

# Последовательность команд, которые мы обычно вводим вручную:
# 1) "1 | 13 | 14 | 15" — создание новой группы из объединения.
# 2) "q"               — выход из gmx make_ndx.
commands = """1 | 13 | 14 | 15
q
"""

# Запускаем gmx make_ndx, передавая ему необходимый ввод через stdin
process = subprocess.Popen(
    ["gmx", "make_ndx", "-f", f"{tmp_folder}/fixed_frame_modif_0.gro", "-o", f"{tmp_folder}/new_index.ndx"],
    stdin=subprocess.PIPE, 
    stdout=subprocess.PIPE, 
    stderr=subprocess.PIPE,
    text=True  # Используем text=True, чтобы можно было передавать строку без encode()
)

# f'gmx make_ndx -f {tmp_folder}/fixed_frame_modif_0.gro -o {tmp_folder}/new_index.ndx'

# Передаём команды и дожидаемся завершения
stdout, stderr = process.communicate(commands)

# Если хотите, можете посмотреть вывод
print("STDOUT:", stdout)
print("STDERR:", stderr)


STDOUT: Going to read 0 old index file(s)
Analysing residue names:
There are:   163    Protein residues
There are:     3      Other residues
There are: 10288      Water residues
There are:     6        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              : 33530 atoms
  1 Protein             :  2614 atoms
  2 Protein-H           :  1301 atoms
  3 C-alpha             :   163 atoms
  4 Backbone            :   489 atoms
  5 MainChain           :   651 atoms
  6 MainChain+Cb        :   803 atoms
  7 MainChain+H         :   813 atoms
  8 SideChain           :  1801 atoms
  9 SideChain-H         :   650 atoms
 10 Prot-Masses         :  2614 atoms
 11 non-Protein         : 30916 atoms
 12 Other               :    46 atoms
 13 JZ4                 :    22 atoms
 14 FR1                 :    11 atoms
 15 FR2                 :    13 atoms
 16 CL                  :     6 atoms
 17 Water               : 308

# Шаг 3: Подготовка файла топологии
У нас появились новые объекты в системе, поэтому их параметры нужно добавить в файл топологии.

К сожалению, у меня пока не получилось автоматический генерировать эти файлы параметров, и пока решено генерировать их по гайду через CGenFF

От пользователя требуется добавить нужные файлы в папку **2_fragments_parametrs**

Пока я делаю это вручную. Не хочу автоматизировать.

# Шаг 4: Подготовка файла tpr

В первую очередь нужно подготовить файл md.mdp (внести в него группу из белка, лиганда и фрагментов).

После этого можно запускать поздание tpr файла

In [58]:
os.system(f'gmx grompp -f {tmp_folder}/new_md.mdp -c {tmp_folder}/fixed_frame_modif_0.gro -p {tmp_folder}/new_topol.top -n {tmp_folder}/new_index.ndx  -o {tmp_folder}/new.tpr')

                :-) GROMACS - gmx grompp, 2024.3-conda_forge (-:

Executable:   /home/daniil/miniconda3/envs/md-intro-tutorial/bin.AVX2_256/gmx
Data prefix:  /home/daniil/miniconda3/envs/md-intro-tutorial
Working dir:  /home/daniil/MD/MM_GBSA/With_FR1/start2end
Command line:
  gmx grompp -f tmp/new_md.mdp -c tmp/fixed_frame_modif_0.gro -p tmp/new_topol.top -n tmp/new_index.ndx -o tmp/new.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'
Generating 1-4 interactions: fudge = 1


Setting the LD random seed to -101590273

Generated 167799 of the 167910 non-bonded parameter combinations

Generated 117432 of the 167910 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'JZ4'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'FR1'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'FR2'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'CL'

turning H bonds into constraints...

The largest distance between excluded atoms is 0.430 nm between atom 1677 and 1685

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.295 nm, buffer size 0.095 nm

Set rlist, assuming 4x4 atom pair-list, to 1.223 nm, buffer size

Number of degrees of freedom in T-Coupling group Protein_JZ4_FR1_FR2 is 6640.71
Number of degrees of freedom in T-Coupling group Water_and_ions is 61743.29

Back Off! I just backed up tmp/new.tpr to tmp/#new.tpr.1#

GROMACS reminds you: "There is just one thing I can promise you about the outer-space program: your tax dollar will go farther." (Wernher von Braun)



0

# Шаг 5: переносим все в нужную папку и запускаем расчет

In [69]:
com = "gmx_MMPBSA -O \
-i mmpbsa.in \
-cs new.tpr \
-ct combined_modif_traj.xtc \
-ci new_index.ndx \
-cg 1 15 \
-cp new_topol.top \
-o FINAL_RESULTS_MMPBSA.dat \
-eo FINAL_RESULTS_MMPBSA.csv"
print(com)

import os


start_dir = os.getcwd()

try:
    os.chdir("3_MMGBSA")
    os.system(com)
finally: 
    os.chdir(start_dir)

gmx_MMPBSA -O -i mmpbsa.in -cs new.tpr -ct combined_modif_traj.xtc -ci new_index.ndx -cg 1 15 -cp new_topol.top -o FINAL_RESULTS_MMPBSA.dat -eo FINAL_RESULTS_MMPBSA.csv


sh: 1: gmx_MMPBSA: not found
